In [13]:
import os
import getpass
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = init_chat_model("gpt-4o-mini", temperature=0.0)

In [14]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
        You are a helpful assistant in extracting specific properties from a passage.
        Extract the desired information from the following passage.

        Only extract the properties mentioned in the 'Sentiment' function.

        Passage: {passage}
    """ 
)

class Sentiment(BaseModel):
    sentiment: str = Field("description: The overall sentiment of the passage, such as 'positive', 'negative', or 'neutral'.")
    confidence: str = Field("description: The confidence in the passage from 0.0 to 10,0.0, where 0.0 is no confidence and 10.0 is complete confidence.")
    aggresion: str = Field("description: The level of aggression in the passage, from 0.0 to 10.0, where 0.0 is no aggression and 10.0 is extreme aggression.")

In [16]:
structured_llm = llm.with_structured_output(Sentiment)

def extract_sentiment(psg: str) -> Sentiment:
    prompt = tagging_prompt.invoke({"passage": psg})
    response = structured_llm.invoke(prompt)
    return response

# Example usage
if __name__ == "__main__":
    psg = "i think that maybe i hate you and maybe i hate everything about you i hate your face i hate your hair i hate your clothes i hate your shoes i hate your car i hate your house i hate your job i hate your friends i hate your family i hate your life"
    sentiment = extract_sentiment(psg)
    print(sentiment)

sentiment='hate' confidence='high' aggresion='high'
